In [892]:
import pandas as pd
import sqlite3
import re
from datetime import datetime, time
from pytz import timezone
import pytz


In [893]:
dat = pd.read_csv("subway_locations_in_us.csv")

In [894]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [895]:
def remove_nan():
    dat.fillna(' ', inplace = True)
    dat.drop(columns = ['Unnamed: 0'], inplace=True, axis=1)

In [896]:
def merge_address():
    dat['address_1'] = dat['address_1'] + ' ' + dat['address_2']
    dat.drop(['address_2'], inplace = True, axis = 1)

In [897]:
def type_conversion():
    dat['postal_code'] = dat['postal_code'].astype('string')
    

In [898]:
def remove_closed():
    for i in range(len(dat)):
        if dat['open_hours'][i].count('Closed Today') == 7:
            dat.drop([i], inplace=True)
        elif dat['open_hours'][i] == ' ':
            dat.drop([i], inplace=True)
        elif dat['open_hours'][i].find('Open 24 Hours') != -1:
            dat['open_hours'][i] = dat['open_hours'][i].replace('Open 24 Hours', '12:00 AM - 12:00 PM')
        
    dat.reset_index(drop = True, inplace=True)
            

In [899]:
remove_nan()
merge_address()
type_conversion()
remove_closed()

/Users/justinchan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [900]:
def open_times():
    hours = []
    for i in range(len(dat)):    
        reg = re.findall("(Closed Today|\d*:\d\d \wM - \d*:\d\d \wM)", dat['open_hours'][i])
        if len(reg) != 7:
            print(reg)
        hours.append(reg)
    return hours

In [901]:
tz = timezone('America/Los_Angeles')
ts = datetime.now(tz).timetuple()
ts

time.struct_time(tm_year=2022, tm_mon=6, tm_mday=17, tm_hour=16, tm_min=54, tm_sec=51, tm_wday=4, tm_yday=168, tm_isdst=1)

In [902]:
dat.head()

,title,address_1,open_hours,city,state,postal_code,latitude,longitude
0,Tikahtnu Commons S/C,1118 N Muldoon Rd Suite 115,Closed Today 10:30 AM - 7:00 PM 10:30 AM - 7:00 PM 10:30 AM - 7:00 PM 10:30 AM - 8:00 PM 10:30 AM - 8:00 PM 10:30 AM - 7:00 PM,Anchorage,AK,99504,61.2037,-149.7447
1,1307 W Northern Lights,1307 W Northern Lights,9:00 AM - 9:00 PM 9:00 AM - 9:00 PM 9:00 AM - 9:00 PM 9:00 AM - 9:00 PM 9:00 AM - 9:00 PM 10:00 AM - 9:00 PM 11:00 AM - 8:00 PM,Anchorage,AK,99503,61.19,-149.8938
2,1330 Huffman Rd,1330 Huffman Rd,8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 9:00 AM - 9:00 PM 10:00 AM - 8:00 PM,Anchorage,AK,99515,61.1194,-149.8974
3,2715 E. Tudor Road,2715 E. Tudor Road,8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 10:00 AM - 10:00 PM 11:00 AM - 10:00 PM,Anchorage,AK,99507,61.1535,-149.8289
4,3104 Seward Hwy,3104 Seward Hwy,8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 8:00 AM - 10:00 PM 10:00 AM - 10:00 PM 11:00 AM - 9:00 PM,Anchorage,AK,99503,61.19,-149.8938


In [903]:
#(X)Function to input data into database
#Figure out how to query from sqlite3 from frontend
#(X)Do we need data modelling for this project?

In [904]:
con = sqlite3.connect('subways.db')
cur = con.cursor()
cur.execute('''CREATE TABLE subway 
                (title text, address_1 text, open_hours text, city text, 
                state text, postal_code varchar, latitude real, longitude real)''')

In [905]:
def insert_db(df_row):
    cur.execute("insert into subway values (?,?,?,?,?,?,?,?)", (df_row[0], df_row[1], df_row[2], df_row[3], df_row[4], df_row[5], df_row[6], df_row[7]))
    con.commit()

In [906]:
for i in range(len(dat)):
    insert_db(dat.loc[i])

In [889]:
con.close()